## _Medical Insurance Costs_

Pada kasus ini, terdapat data tentang informasi kesehatan dan biaya yang harus dikeluarkan oleh asuransi kesehatan. Infomasi terkait dengan data _medical insurance cost_ adalah sebagai berikut,

1. Age: Usia penerima manfaat
2. Sex: Gender penerima manfaat (_male_, _femele_)
3. Bmi : Body Mass Index
4. Children: Jumlah anak/tanggungan yang dicover oleh pihak asuransi
5. Smoker: Status perokok (_yes_, _no_)
6. Region: Wilayah tempat tinggal penerima manfaat
7. Charges: Biaya yang dikeluarkan oleh asuransi

In [1]:
# Selayang pandang data Medical Insurance Costs
import pandas as pd

df = pd.read_csv('data/insurance.csv')

display(df.head())

display(df.corr())

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


,age,bmi,children,charges
age,1.000000,0.109272,0.042469,0.299008
bmi,0.109272,1.000000,0.012759,0.198341
children,0.042469,0.012759,1.000000,0.067998
charges,0.299008,0.198341,0.067998,1.000000


### Tantangan

Buatlah model regresi untuk memprediksi biaya yang harus dikeluarkan oleh pihak asuransi berdasarkan data. Validasi performa model regresi Anda dengan nilai ***R-squared ($R^2$)***

#### _Tasks_

1. Pastikan semua variabel kategorial diolah dengan baik. (Gunakan fitur mapping pada pandas)
2. Cek kondisi multicollinearity untuk semua variabel independen. Jika ada, antar variabel apakah itu?
3. Pastikan model menggunakan variabel yang tidak memiliki nilai multicollinearity yang tinggi
4. (Hints) Anda dapat menggunakan nilai ***Variance Inflation Factor (VIF)*** untuk mengetahui tingkat multicollinearity pada sebuah variabel independent.
5. Evaluasi model yang Anda buat dengan nilai $R^2$
6. Simpulkan, variabel independen apa saja yang dapat digunakan untuk menghasilkan model regresi yang baik pada kasus _medical insurance costs_?

#### (Hints) Interpretasi Nilai VIF

- 1 - variabel indenpenden tidak memiliki korelasi dengan variabel independen yang lain
- 1 < VIF < 5 - variabel independen sedikit memiliki korelasi dengan variabel independen yang lain
- VIF > 5 - variabel independen memiliki korelasi yang kuat dengan variabel independen lainnya
- VIF > 10 - variabel independen miliki korelasi yang sangat kuat dengan variabel independen dan perlu diperhatikan lebih lanjut

#### (Hints) Implementasi Perhitungan VIF

VIF dapat dihitung secara langsung dengan menggunakan library dari `statsmodels`

#### (Hints) Scatterplot Korelasi Antar Variabel

![var_cor](assets/var_corr.png)

In [2]:
# Import Library

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [3]:
# Menampilkan tabel

tabel = pd.read_csv('data/insurance.csv')
# Head untuk data 5 teratas dan corr untuk korelasi antar atribut
display(tabel.head())

display(tabel.corr())

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


,age,bmi,children,charges
age,1.000000,0.109272,0.042469,0.299008
bmi,0.109272,1.000000,0.012759,0.198341
children,0.042469,0.012759,1.000000,0.067998
charges,0.299008,0.198341,0.067998,1.000000


In [4]:
# SOAL 1

new_sex = {
    'male': 1,
    'female': 0
}

new_smoker ={
    'yes' : 1,
    'no' : 0
}

new_region ={
    'northwest' : 0,
    'southwest' : 1,
    'southeast' : 2,
    'northeast' : 3 
}

# Untuk encode label baru
# mapping berfungsi untuk mengubah variabel kategorial ke angka supaya bisa dikenali python
tabel['sex'] = tabel['sex'].map(new_sex)
tabel['smoker'] = tabel['smoker'].map(new_smoker)
tabel['region'] = tabel['region'].map(new_region)


# Menampilkan tabel dengan urutan teratas dan jumlahnya 5 karena di head tidak di definisikan nilainya
tabel.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,1,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,0,21984.47061
4,32,1,28.880,0,0,0,3866.85520


In [8]:
# SOAL 2, 3, 4
# Dengan fitur VIF dapat mengetahui tingkat multicollinearity
# setalah dilakukan VIF, yang tidak memiliki nilai multicollinearity yang tinggi yaitu sex, children, smoker, region


from statsmodels.stats.outliers_influence import variance_inflation_factor
  
# set variabel independen
new_table = tabel[['age', 'sex' ,'bmi' ,'children', 'smoker', 'region']]
  
# VIF dataframe
vif = pd.DataFrame()
vif["feature"] = new_table.columns
  
# menghitung VIF untuk setiap dataframe
vif["VIF"] = [variance_inflation_factor(new_table.values, i)
                          for i in range(len(new_table.columns))]
  
print(vif)

    feature       VIF
0       age  7.588734
1       sex  2.002652
2       bmi  9.938670
3  children  1.799980
4    smoker  1.261195
5    region  2.789478


In [9]:
# Peng-copyan tabel/dataframe sebelum dilakukan penghapusan age dan bmi
tabel_baru = tabel.copy()
tabel_baru.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,1,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,0,21984.47061
4,32,1,28.880,0,0,0,3866.85520


In [10]:
# Hapus bmi dan age
tabel_baru = tabel_baru.drop(columns=['age', 'bmi'])
tabel_baru.head()

,sex,children,smoker,region,charges
0,0,0,1,1,16884.92400
1,1,1,0,2,1725.55230
2,1,3,0,2,4449.46200
3,1,0,0,0,21984.47061
4,1,0,0,0,3866.85520


In [11]:
# SOAL 5

# Fitur r_square untuk menentukan akurasi dari variabel independen

# x untuk data kolom dan y untuk kolom terakhir
X = tabel_baru.iloc[:, :-1].values
y = tabel_baru.iloc[:, 4].values

In [12]:
# 4 variabel digunakan untuk melakukan train_test_split 
# Dengan parameter x,  y, test_size 0.2, dan random_state 40

xm_train, xm_test, ym_train, ym_test = train_test_split(X, y, test_size=0.2, random_state=60)

In [13]:
# Mengganti bentuk y menjadi 2 dimensi

y = y.reshape(len(y), 1)
y.shape

(1338, 1)

In [14]:
# Kemudian dilakukan fit pada xm_train dan ym_train
# dan di predict dengan xm_test

mlr = LinearRegression()
mlr.fit(xm_train, ym_train)

ym_pred = mlr.predict(xm_test)

In [15]:
# Proses penggabungan

gabung = np.concatenate((ym_test, ym_pred))
gabung

array([ 5373.36425   , 10338.9316    ,  2709.1119    , 44260.7499    ,
       10560.4917    , 12644.589     ,  6311.952     , 11520.09985   ,
        8410.04685   , 29523.1656    , 19107.7796    , 38711.        ,
       12269.68865   , 18804.7524    ,  8062.764     , 12949.1554    ,
       28868.6639    , 11305.93455   ,  1615.7667    , 10269.46      ,
        5266.3656    ,  7147.105     ,  2459.7201    ,  7640.3092    ,
       12224.35085   ,  9377.9047    , 22218.1149    , 10264.4421    ,
       11436.73815   , 30063.58055   , 11657.7189    , 13770.0979    ,
        2473.3341    ,  5124.1887    ,  9991.03765   , 48173.361     ,
        3943.5954    ,  2154.361     ,  8017.06115   ,  1728.897     ,
       12146.971     , 28340.18885   ,  2331.519     , 23045.56616   ,
        9361.3268    ,  6402.29135   , 26125.67477   , 29186.48236   ,
       12622.1795    ,  6128.79745   ,  4922.9159    ,  4877.98105   ,
        4618.0799    ,  9715.841     , 13063.883     , 12523.6048    ,
      

In [16]:
# Prediksi menggunakan r2_score

r2_mlr = r2_score(ym_test, ym_pred)
print(r2_mlr)

0.5223291105621384


In [17]:
# SOAL 6

# Pengecekan akurasi menggunakan semua kolom

# x untuk data kolom dan y untuk kolom terakhir
X2 = tabel.iloc[:, :-1].values
y2 = tabel.iloc[:, 6].values

In [18]:
# 4 variabel digunakan untuk melakukan train_test_split 
# Dengan parameter x,  y, test_size 0.2, dan random_state 40

xm_train2, xm_test2, ym_train2, ym_test2 = train_test_split(X2, y2, test_size=0.2, random_state=50)

In [19]:
# Mengganti bentuk y menjadi 2 dimensi

y2 = y2.reshape(len(y), 1)
y2.shape

(1338, 1)

In [20]:
# Kemudian dilakukan fit pada xm_train dan ym_train
# dan di predict dengan xm_test

mlr2 = LinearRegression()
mlr2.fit(xm_train2, ym_train2)

ym_pred2 = mlr2.predict(xm_test2)

In [21]:
# Proses penggabungan

gabung2 = np.concatenate((ym_test2, ym_pred2))
gabung2

array([ 5.97683110e+03,  5.84691760e+03,  1.38311152e+04,  9.62592000e+03,
        2.68094930e+03,  4.78967913e+04,  1.82234512e+04,  7.41947790e+03,
        3.73262510e+03,  1.22228983e+04,  7.05002130e+03,  2.19786769e+04,
        6.28223500e+03,  3.77018768e+04,  7.04672220e+03,  1.20323260e+04,
        1.31126048e+04,  4.23989265e+03,  1.23338280e+04,  3.41032400e+03,
        1.72778500e+03,  4.46411974e+04,  1.71284261e+04,  6.11235295e+03,
        4.52947700e+03,  1.05945016e+04,  6.40229135e+03,  4.61511245e+04,
        1.71102680e+03,  1.70470015e+03,  4.58632050e+04,  4.68779700e+03,
        1.50197601e+04,  3.18051010e+03,  3.86120965e+03,  3.44306400e+03,
        2.71179938e+04,  2.70924395e+03,  1.34511220e+04,  4.79280300e+04,
        2.35630162e+04,  6.71019190e+03,  1.42350720e+04,  1.40011338e+04,
        2.72184372e+04,  1.33905590e+04,  4.10342214e+04,  2.02017700e+03,
        1.42561928e+04,  2.12321823e+04,  4.86755177e+04,  6.98669700e+03,
        4.14973600e+03,  

In [22]:
# Prediksi menggunakan r2_score

r2_mlr2 = r2_score(ym_test2, ym_pred2)
print(r2_mlr2)

0.7835627749480735
